In [1]:
import pyviz3d.visualizer as viz
import numpy as np
import math
import open3d as o3d
import pickle

import numpy as np
import pyviz3d.visualizer as viz
# import pyviz3d.src.blender_tools as blt 
import plyfile
import open3d as o3d
import os
from sklearn.neighbors import NearestNeighbors

#from Yang
def read_ply_data(filename):
    filename_in = filename
    file = open(filename_in, 'rb')
    ply_data = plyfile.PlyData.read(file)
    file.close()
    x = ply_data['vertex']['x']
    y = ply_data['vertex']['y']
    z = ply_data['vertex']['z']
    red = ply_data['vertex']['red']
    green = ply_data['vertex']['green']
    blue = ply_data['vertex']['blue']
    object_id = ply_data['vertex']['objectId']
    global_id = ply_data['vertex']['globalId']
    nyu40_id = ply_data['vertex']['NYU40']
    eigen13_id = ply_data['vertex']['Eigen13']
    rio27_id = ply_data['vertex']['RIO27']
 
    vertices = np.empty(len(x), dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'),  ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'),
                                                     ('objectId', 'h'), ('globalId', 'h'), ('NYU40', 'u1'), ('Eigen13', 'u1'), ('RIO27', 'u1')])
    
    vertices['x'] = x.astype('f4')
    vertices['y'] = y.astype('f4')
    vertices['z'] = z.astype('f4')
    vertices['red'] = red.astype('u1')
    vertices['green'] = green.astype('u1')
    vertices['blue'] = blue.astype('u1')
    vertices['objectId'] = object_id.astype('h')
    vertices['globalId'] = global_id.astype('h')
    vertices['NYU40'] = nyu40_id.astype('u1')
    vertices['Eigen13'] = eigen13_id.astype('u1')
    vertices['RIO27'] = rio27_id.astype('u1')
    return vertices



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def generate_ball_pcl(position, radius, color, resolution=5):
    # generate ball
    points = []
    ## generate ball points
    for i in range(0, 360, resolution):
        for j in range(0, 360, resolution):
            x = radius * np.cos(np.deg2rad(i)) * np.sin(np.deg2rad(j))
            y = radius * np.sin(np.deg2rad(i)) * np.sin(np.deg2rad(j))
            z = radius * np.cos(np.deg2rad(j))
            points.append([x, y, z])
    points = np.array(points)
    ## translate ball
    points += position
    colors = np.array([color for _ in range((points.shape[0]))])
    return points, colors

def generate_camera_frustum(vis, cam_extrinsics, target_scene_center = np.array([0, 0, 0]),
                            cam_color = [0, 0, 0],
                            img_width_m = 0.7, cam_depth_range = 0.8, 
                            point_size = 0.04, img_size = (540, 960)):
    image_h = img_size[1]
    image_w = img_size[0]
    img_height_m = img_width_m * image_h / image_w
    
    # get points in camera frustum
    image_center = np.array([0, 0, 0])
    image_corners =  np.array([
        [img_width_m/2.0, img_height_m/2.0, cam_depth_range],
        [-img_width_m/2.0, img_height_m/2.0, cam_depth_range],
        [-img_width_m/2.0, -img_height_m/2.0, cam_depth_range],
        [img_width_m/2.0, -img_height_m/2.0, cam_depth_range],
    ])
    T_rescan2ref_right, T_camera2rescan = cam_extrinsics[0], cam_extrinsics[1]
    # transform to world coord
    image_center =  image_center @ (T_camera2rescan[:3, :3]).T  + T_camera2rescan[:3, 3].T
    image_corners = image_corners @ (T_camera2rescan[:3, :3]).T  + T_camera2rescan[:3, 3].T
    image_center =  image_center @ T_rescan2ref_right[:3, :3]  + T_rescan2ref_right[3, :3]
    image_corners = image_corners @ T_rescan2ref_right[:3, :3]  + T_rescan2ref_right[3, :3]
    image_center = image_center + target_scene_center.reshape(1, 3)
    image_corners = image_corners + target_scene_center.reshape(1, 3)
    # get lines in camera frustum
    line_ps_start = []
    line_ps_end = []
    black_color = np.array([0.0, 0.0, 0.0])
    vis.add_points('img_center', image_center.reshape(-1, 3), black_color.reshape(-1, 3), point_size=100, resolution=15, visible=True)    
    for i in range(4):
        # line_ps_start.append(image_center.reshape(3))
        # line_ps_end.append(image_corners[i].reshape(3))
        edges = []
        edges.append(image_center.reshape(3))
        edges.append(image_corners[i].reshape(3))
        # obj = cylinder_between(x1, y1, z1, x2, y2, z2, properties['edge_width'] * 2, properties['color'])
        # create_mat(obj)
        vis.add_polyline(f'img_center_corner{i}', np.array(edges), color=np.array([0, 0, 0]), edge_width=0.015)
        vis.add_points(f'corner{i}', image_center.reshape(-1,3), black_color.reshape(-1, 3), point_size=25, resolution=15, visible=True)    
    for i in range(4):
        # line_ps_start.append(image_corners[i].reshape(3))
        # line_ps_end.append(image_corners[(i+1)%4].reshape(3))
        edges = []
        edges.append(image_corners[i].reshape(3))
        edges.append(image_corners[(i+1)%4].reshape(3))
        vis.add_polyline(f'center{[i]}_corner{i+1}', np.array(edges), color=np.array([0, 0, 0]), edge_width=0.015)
    # line_ps_start = np.array(line_ps_start)
    # line_ps_end = np.array(line_ps_end)
    # # generate point balls
    # points_center, colors_center = generate_ball_pcl(image_center, point_size, cam_color)
   

def save_visualize(data_dir, scan_id, mesh_file, out_folder, img_pose = None, img_size = (540, 960), 
                   node_size = 200, matched_node_size = 400, matched_ids = np.array([]),
                   render_cam_pose = [-0.265198, -0.411423, 7.11054],
                   render_cam_look = [0.0, 0.0, 0.0], obj_count_th = 1000,
                   blender_path = None):
    scene = o3d.io.read_point_cloud(mesh_file)

    scene_meta = read_ply_data(mesh_file)

    scene.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

    point_normals = np.asarray(scene.normals)
    point_positions = np.asarray(scene.points)
    points_center = np.mean(point_positions, axis=0)
    point_positions -= points_center
    point_colors = np.asarray(scene.colors)
    objectIds = scene_meta['objectId']
    
    # filter out points with color (0, 0, 0)
    mask = np.any(point_colors > 0, axis=1)
    point_positions = point_positions[mask]
    point_normals = point_normals[mask]
    point_colors = point_colors[mask]
    objectIds = objectIds[mask]
    ## filter out points with number of points less than 100
    unique_objectIds, counts = np.unique(objectIds, return_counts=True)
    mask = np.isin(objectIds, unique_objectIds[counts > obj_count_th])
    point_positions = point_positions[mask]
    point_normals = point_normals[mask]
    point_colors = point_colors[mask]
    objectIds = objectIds[mask]
    
    print("point_positions.shape", point_positions.shape)
    print("point_normals.shape", point_normals.shape)
    print("point_colors.shape", point_colors.shape)
    print("objectIds.shape", objectIds.shape)

    # Find instance centers and colors
    
    unique_objectIds = np.unique(objectIds)
    obj_centers = []
    obj_colors = []
    obj_ids = []

    
    for obj_id in unique_objectIds:
        obj_points = point_positions[objectIds == obj_id]
        obj_center = np.mean(obj_points, axis=0)
        obj_color = point_colors[objectIds == obj_id][0, :]
        
        obj_positions_relative = obj_points - obj_center
        dist_norm = np.linalg.norm(obj_positions_relative, axis=1)
        nearest_idx = np.argsort(dist_norm)[0]
        obj_patch_pos = obj_points[nearest_idx]
        obj_center = obj_patch_pos
        
        obj_centers.append(obj_center)
        obj_colors.append(obj_color)
        obj_ids.append(obj_id)
    obj_centers = np.concatenate(obj_centers).reshape(-1, 3)
    obj_colors = np.concatenate(obj_colors).reshape(-1, 3)
    mask = obj_centers[:, 2] < 0.8
    obj_centers = obj_centers[mask]
    obj_colors = obj_colors[mask]
    obj_ids = np.array(obj_ids)[mask]
    
    vis_nodes = viz.Visualizer(position=render_cam_pose, look_at =  render_cam_look, focal_length=28)
    vis_pcs = viz.Visualizer(position=render_cam_pose, look_at =  render_cam_look, focal_length=28)
    vis_pcs_nodes = viz.Visualizer(position=render_cam_pose, look_at =  render_cam_look, focal_length=28)
    pcs_mask = point_positions[:,2] < 0.8
    for idx, obj_id in enumerate(obj_ids):
        obj_center = obj_centers[idx]
        obj_color = obj_colors[idx]
        obj_size = node_size if obj_id not in matched_ids else matched_node_size

        vis_nodes.add_points('obj_{}'.format(obj_id), obj_center.reshape(-1,3), 
                            obj_color.reshape(-1,3) * 255, point_size=obj_size, resolution=15, visible=True)  
        vis_pcs_nodes.add_points('obj_{}'.format(obj_id), obj_center.reshape(-1,3), 
                            obj_color.reshape(-1,3) * 255, point_size=obj_size, resolution=15, visible=True)  

        obj_mask = np.logical_and( objectIds == obj_id, pcs_mask)
        
        if len(matched_ids) == 0:
            alpha = 1.0
            vis_pcs.add_points('pcs_obj_{}'.format(obj_id), point_positions[obj_mask], point_colors[obj_mask] * 255, point_normals[obj_mask], 
                point_size=50, visible=True, alpha=alpha)
        else:
            alpha = 0.1 if obj_id not in matched_ids else 1.0 
            vis_pcs.add_points('pcs_obj_{}'.format(obj_id), point_positions[obj_mask], point_colors[obj_mask] * 255, point_normals[obj_mask], 
                point_size=50, visible=True, alpha=alpha)
            
        vis_pcs_nodes.add_points('pcs_obj_{}'.format(obj_id), point_positions[obj_mask], point_colors[obj_mask] * 255, point_normals[obj_mask], 
            point_size=50, visible=True, alpha=0.1) 
                
    # nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(obj_centers)
    # distances, indices = nbrs.kneighbors(obj_centers)
    # for i in range(indices.shape[0]):
    #     for jj, j in enumerate(indices[i].tolist()):
    #         if distances[i][jj] < 2.0 and distances[i][jj] > 0.001:
    #             edges = []
    #             edges.append(obj_centers[i])
    #             edges.append(obj_centers[j])
    #             vis_nodes.add_polyline(f'polyline{i}{j}', np.array(edges), color=np.array([250.0, 250.0, 250.0]), edge_width=0.02)
    #             vis_pcs_nodes.add_polyline(f'polyline{i}{j}', np.array(edges), color=np.array([250.0, 250.0, 250.0]), edge_width=0.02)


    #make every edge visible in the graph

    print("object ids", obj_ids)

    path = os.path.join(data_dir, "files/orig/data/" "{}.pkl".format(scan_id))
    with open(path, 'rb') as file:
        data = pickle.load(file)

    edges = data["edges"]
    
    for edge in edges:
        i, j = edge
        
        if i != 0 and j != 0:
            idx = np.where(obj_ids == i)[0]

            jdx = np.where(obj_ids == i)[0]

            if  len(idx) != 0 and len(jdx) != 0:

                # Fetch the coordinates from obj_centers based on indices i and j
                coord_i = obj_centers[idx]
                coord_j = obj_centers[jdx]
                
                # Add the edge to the list
                ed = []
                ed.append([coord_i, coord_j])
                
                # Add polyline to vis_nodes
                vis_nodes.add_polyline(f'polyline{i}{j}', np.array(ed), color=np.array([250.0, 250.0, 250.0]), edge_width=0.02)
                
                # Add polyline to vis_pcs_nodes if needed (assuming it has similar add_polyline method)
                vis_pcs_nodes.add_polyline(f'polyline{i}{j}', np.array(ed), color=np.array([250.0, 250.0, 250.0]), edge_width=0.02)


    
    

    
    
    if img_pose is not None:
        # line_ps_start, line_ps_end, points_frustum, colors_frustum = \
        #     generate_camera_frustum(img_pose, img_size=img_size, img_width_m=0.4)
        # # add image frustum
        # vis_pcs.add_points(
        #     'frustum', 
        #     positions=points_frustum,
        #     colors=colors_frustum)
        # vis_pcs.add_lines(
        #     'frustum_lines', 
        #     line_ps_start, line_ps_end)
        generate_camera_frustum( vis_pcs, img_pose, img_size=img_size, img_width_m=0.6, cam_depth_range=0.4, target_scene_center=-points_center)
        
    # When we added everything we need to the visualizer, we save it.
    out_folder_pcs = os.path.join(out_folder, "pcs")
    out_folder_nodes = os.path.join(out_folder, "nodes")
    out_folder_pcs_nodes = os.path.join(out_folder, "pcs_nodes")

    #make sure the directories exist
    for dir_path in [out_folder_pcs, out_folder_nodes, out_folder_pcs_nodes]:
        try:
            os.makedirs(dir_path, exist_ok=True)
            
        except Exception as e:
            print(f"Failed to create directory {dir_path}: {e}")

    blender_args = {
    "executable_path": blender_path,
    "output_prefix" : False
    }
            
    vis_pcs.save(path =out_folder_pcs, port= 8080,blender_args= blender_args, verbose= True)
    vis_nodes.save(path = out_folder_nodes, port= 8080, blender_args= blender_args, verbose= True )
    vis_pcs_nodes.save(path = out_folder_pcs_nodes, port= 8080,blender_args= blender_args, verbose= True)
    # return {
    #     'img_pose': img_pose,
    #     'line_ps_start': line_ps_start,
    #     'line_ps_end': line_ps_end,
    #     'points_frustum': points_frustum,
    #     'colors_frustum': colors_frustum,
        
    # }

In [3]:
# 20c993b3-698f-29c5-859c-dca8ddecf220
scan_id = "0ad2d384-79e2-2212-9b18-72b44eb5463f"

mesh_file = os.path.join('/local/home/ekoller/R3Scan/scenes', scan_id, "labels.instances.align.annotated.v2.ply")
data_dir = "/local/home/ekoller/R3Scan"
out_folder = os.path.join("/local/home/ekoller/visscenegraph",scan_id)
blender_path = "/local/home/ekoller/visscenegraph/blender-4.1.1-linux-x64/blender"
save_visualize(data_dir, scan_id, mesh_file, out_folder, img_size=(960, 540), 
               node_size=200, matched_node_size=500,
               render_cam_pose = [-0.5, -2.5, 2.8], render_cam_look = [0.0, 0.0, -0.3], obj_count_th=100, blender_path=blender_path)

point_positions.shape (52064, 3)
point_normals.shape (52064, 3)
point_colors.shape (52064, 3)
objectIds.shape (52064,)
object ids [ 1  2  3  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 24 27 33 35 37
 38 39 40 44 45 46]

************************************************************************
1) Start local server:
    cd /local/home/ekoller/visscenegraph/0ad2d384-79e2-2212-9b18-72b44eb5463f/pcs; python -m http.server 8080
2) Open in browser:
    http://localhost:8080
************************************************************************
Blender 4.1.1 (hash e1743a0317bc built 2024-04-15 23:47:45)
<bpy_struct, Object("Cube") at 0x7ff6a3939708>
<bpy_struct, Object("Light") at 0x7ff6a3939c08>
<bpy_struct, Object("Camera") at 0x7ff6a3939208>
Camera_0 {'type': 'camera', 'position': [-0.5, -2.5, 2.799999952316284], 'look_at': [0.0, 0.0, -0.30000001192092896], 'up': [0.0, 0.0, 1.0], 'focal_length': 28.0, 'animation': False}
pcs_obj_1 {'type': 'points', 'visible': True, 'alpha': 1.0, 'sha

In [41]:
#prints out data so that I get an overview over what is inside of it
import pickle

path = os.path.join('/local/home/ekoller/R3Scan/files/orig/data/', "{}.pkl".format(scan_id))
with open(path, 'rb') as file:
    data = pickle.load(file)

if isinstance(data, dict):
    for key, value in data.items():
        print(f'Key: {key}')
        print(f'Value: {value}')
        print('---')


Key: scan_id
Value: 0ad2d384-79e2-2212-9b18-72b44eb5463f
---
Key: objects_id
Value: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 24 25 27
 31 33 35 37 38 39 40 44 45 46]
---
Key: global_objects_id
Value: [188  37 503  82 503 520 129 503 391 503 110 155 455  85 155 295  85  15
  15 503 489 236 155 288   1 243 391 338 338 476 476 263 263 158]
---
Key: objects_cat
Value: [188  37 503  82 503 520 129 503 391 503 110 155 455  85 155 295  85  15
  15 503 489 236 155 288   1 243 391 338 338 476 476 263 263 158]
---
Key: triples
Value: [[2, 1, 15], [3, 1, 14], [4, 3, 14], [4, 5, 14], [4, 8, 14], [4, 10, 14], [5, 1, 14], [6, 5, 14], [7, 5, 17], [8, 1, 14], [9, 8, 14], [10, 1, 14], [11, 1, 15], [12, 3, 17], [13, 1, 15], [14, 1, 15], [15, 46, 21], [16, 1, 15], [17, 1, 15], [18, 1, 16], [19, 1, 16], [21, 13, 15], [24, 1, 15], [27, 10, 14], [33, 13, 15], [35, 1, 15], [37, 2, 16], [38, 2, 16], [39, 2, 16], [40, 2, 16], [44, 5, 17], [45, 5, 17], [46, 1, 15], [46, 10, 14], [46, 8, 1